In [5]:
from keras.models import load_model
import cv2 as cv
import numpy as np
import os

In [6]:
# Load Folders in data_test
data_test_dir = [dir for dir in os.listdir("./data_test/")]

In [7]:
np.set_printoptions(suppress=True)

# Load the model and labels
model = load_model("keras_Model.h5", compile=False)
class_names = open("labels.txt", "r").readlines()

In [8]:
stop = False
for video_dir in range(len(data_test_dir)):
    if stop:
        break
    for frame in range(100): 
    # Loading sample image
        img_dir = [i for i in sorted(os.listdir(f"./data_test/{data_test_dir[video_dir]}"))]
        img_path = f"./data_test/{data_test_dir[video_dir]}/{img_dir[frame]}"
        img = cv.imread(img_path)

        # Create a bounding boxes based on annotations file
        boxes_lists = []
        label_texts = []
        annotation_file = f"./annotations/{data_test_dir[video_dir]}.csv"

        with open (annotation_file, "r") as x:
            for lines in x.readlines():
                temp = lines.split(",")
                try:
                    if int(temp[1]) == frame:
                        box = (int(temp[2]), int(temp[3]), int(temp[4]), int(temp[5]))
                        boxes_lists.append(box)

                        # Run the model and get the prediction
                        temp_img = img.copy()
                        img_crop = temp_img[box[1]:box[1]+box[3], box[0]:box[0]+box[2]]
                        gray_img = cv.cvtColor(img_crop, cv.COLOR_BGR2GRAY)
                        # apply gaussian blur
                        gauss_img = cv.GaussianBlur(gray_img, (5,5), 0)
                        # apply Canny Edge
                        proc_img = cv.Canny(image=gauss_img, threshold1=25, threshold2=150)
                        #Resize
                        img_res = cv.resize(proc_img, (224, 224), interpolation=cv.INTER_AREA)
                        #For normalization
                        img_norm = np.asarray(img_res, dtype=np.float32)
                        img_norm = np.expand_dims(img_norm, axis=-1)
                        img_norm = np.repeat(img_norm, 3, axis=-1)
                        img_norm = img_norm.reshape(1, 224, 224, 3)
                        img_toPred = (img_norm / 127.5) - 1
                        #predicts className and confidence score
                        prediction = model.predict(img_toPred, verbose=0)
                        index = np.argmax(prediction)
                        class_name = class_names[index]
                        confidence_score = prediction[0][index] #how sure na correct ang className
                        label = f"{class_name[2:-1]}@{confidence_score*100:.2f}%"
                        label_texts.append(label)
                except Exception as e:
                    pass

        for i in range(len(boxes_lists)):
            #BB with text
            cv.rectangle(img, boxes_lists[i], (0,0,255), 2)
            cv.putText(img, label_texts[i], (boxes_lists[i][0], boxes_lists[i][1]), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)

        img_rescaled = cv.resize(img, (1280, 720))

        cv.imshow("test_img", img_rescaled)
        key = cv.waitKey(20)
        if key == 27:
            stop = True
            break
        elif key == 48:
            break

cv.destroyAllWindows()